In [1]:
import pandas as pd
from torch import nn
import torch

In [2]:
data = pd.read_csv(r"C:\Users\Vansh\Desktop\projects\Sap marathon\Crop_recommendation.csv")

In [3]:
data

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice
...,...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507,coffee
2196,99,15,27,27.417112,56.636362,6.086922,127.924610,coffee
2197,118,33,30,24.131797,67.225123,6.362608,173.322839,coffee
2198,117,32,34,26.272418,52.127394,6.758793,127.175293,coffee


In [4]:
cr = data["label"].unique()
fer = ['barley', 'cotton',
       'ground nuts', 'maize', 'millets',
       'oil seeds', 'paddy', 'pulses',
       'sugarcane', 'tobacco', 'wheat']


In [5]:
cr

array(['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
       'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
       'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
       'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee'],
      dtype=object)

In [6]:
data2 = data.copy()

In [7]:
data2["label"] = data2["label"].replace('rice', 0)
data2["label"] = data2["label"].replace('maize', 1)
data2["label"] = data2["label"].replace('chickpea', 2)
data2["label"] = data2["label"].replace('kidneybeans', 3)
data2["label"] = data2["label"].replace('pigeonpeas', 4)
data2["label"] = data2["label"].replace('mothbeans', 5)
data2["label"] = data2["label"].replace('mungbean', 6)
data2["label"] = data2["label"].replace('blackgram',7)
data2["label"] = data2["label"].replace('lentil', 8)
data2["label"] = data2["label"].replace('pomegranate', 9)
data2["label"] = data2["label"].replace('banana', 10)
data2["label"] = data2["label"].replace('mango', 11)
data2["label"] = data2["label"].replace('grapes', 12)
data2["label"] = data2["label"].replace('watermelon', 13)
data2["label"] = data2["label"].replace('muskmelon', 14)
data2["label"] = data2["label"].replace('apple', 15)
data2["label"] = data2["label"].replace('orange', 16)
data2["label"] = data2["label"].replace('papaya', 17)
data2["label"] = data2["label"].replace('coconut', 18)
data2["label"] = data2["label"].replace('cotton', 19)
data2["label"] = data2["label"].replace('jute', 20)
data2["label"] = data2["label"].replace('coffee', 21)


data2

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,0
1,85,58,41,21.770462,80.319644,7.038096,226.655537,0
2,60,55,44,23.004459,82.320763,7.840207,263.964248,0
3,74,35,40,26.491096,80.158363,6.980401,242.864034,0
4,78,42,42,20.130175,81.604873,7.628473,262.717340,0
...,...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507,21
2196,99,15,27,27.417112,56.636362,6.086922,127.924610,21
2197,118,33,30,24.131797,67.225123,6.362608,173.322839,21
2198,117,32,34,26.272418,52.127394,6.758793,127.175293,21


In [8]:
X = data2.drop('label',axis = 1)
Y = data2['label']

In [9]:
X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(Y, dtype=torch.long)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.20,random_state=0)

In [11]:
crop_Ann = nn.Sequential(
      nn.Linear(7,64),   
      nn.ReLU(),         
      nn.Linear(64,64),  
      nn.ReLU(),         
      nn.Linear(64,22),   
        )

In [12]:
import tqdm
from tqdm import trange
import torch.optim as optim
 
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(crop_Ann.parameters(), lr=0.001)

In [15]:
# Training parameters
n_epochs = 20
batch_size = 10
batches_per_epoch = len(X_train) // batch_size

for epoch in range(n_epochs):
    for i in range(batches_per_epoch):
        # Take a batch
        start = i * batch_size
        X_batch = X_train[start:start+batch_size]
        y_batch = Y_train[start:start+batch_size]
        
        # Forward pass
        y_pred = crop_Ann(X_batch)
        
        # Convert y_batch to LongTensor if needed
        y_batch = y_batch.long()
        
        # Compute loss and accuracy
        loss = loss_fn(y_pred, y_batch)
        acct = (torch.argmax(y_pred, 1) == y_batch).float().mean() 
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch} train: Cross-entropy={float(loss)}, Accuracy={float(acct)}")
    
    # Evaluate on validation data
    with torch.no_grad():  # No need to compute gradients during evaluation
        y_pred_test = crop_Ann(X_test)
        Y_test_tensor = Y_test.long()  # Convert Y_test to LongTensor if needed
        ce = loss_fn(y_pred_test, Y_test_tensor)
        acc = (torch.argmax(y_pred_test, 1) == Y_test_tensor).float().mean()
        print(f"Epoch {epoch} validation: Cross-entropy={float(ce)}, Accuracy={float(acc)}")



Epoch 0 train: Cross-entropy=0.613364577293396, Accuracy=0.699999988079071
Epoch 0 validation: Cross-entropy=0.31028810143470764, Accuracy=0.8863636255264282
Epoch 1 train: Cross-entropy=0.2935003936290741, Accuracy=1.0
Epoch 1 validation: Cross-entropy=0.22548595070838928, Accuracy=0.9295454621315002
Epoch 2 train: Cross-entropy=0.20182058215141296, Accuracy=1.0
Epoch 2 validation: Cross-entropy=0.1940087527036667, Accuracy=0.9409090876579285
Epoch 3 train: Cross-entropy=0.14826011657714844, Accuracy=1.0
Epoch 3 validation: Cross-entropy=0.1757650524377823, Accuracy=0.9454545378684998
Epoch 4 train: Cross-entropy=0.13358034193515778, Accuracy=1.0
Epoch 4 validation: Cross-entropy=0.1616898626089096, Accuracy=0.9454545378684998
Epoch 5 train: Cross-entropy=0.09248636662960052, Accuracy=1.0
Epoch 5 validation: Cross-entropy=0.15248127281665802, Accuracy=0.9454545378684998
Epoch 6 train: Cross-entropy=0.06588320434093475, Accuracy=1.0
Epoch 6 validation: Cross-entropy=0.14207270741462708

In [15]:
data2

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,0
1,85,58,41,21.770462,80.319644,7.038096,226.655537,0
2,60,55,44,23.004459,82.320763,7.840207,263.964248,0
3,74,35,40,26.491096,80.158363,6.980401,242.864034,0
4,78,42,42,20.130175,81.604873,7.628473,262.717340,0
...,...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507,21
2196,99,15,27,27.417112,56.636362,6.086922,127.924610,21
2197,118,33,30,24.131797,67.225123,6.362608,173.322839,21
2198,117,32,34,26.272418,52.127394,6.758793,127.175293,21


In [16]:
import torch
torch.save(crop_Ann, r'C:\Users\Vansh\Desktop\crop1.pth')

In [17]:
import torch

mod = torch.load(r'C:\Users\Vansh\Desktop\crop1.pth')
ut1 = [90.0, 42.0, 43.0,20.879744,82.002744, 6.502985,202.935536]

# Convert ut1 to LongTensor

ut2 = torch.tensor(ut1)

# Assuming the last layer is the output layer
output = mod(ut2)

In [18]:
output

tensor([ 13.3872,  -1.7968,  -7.2879, -12.3978,  -3.2228, -12.8115, -14.9239,
        -22.0175, -34.8098,  -1.4393,  -7.0617,  -4.6661, -14.9548,  -6.7838,
        -29.1311,   2.2138,  -2.8374,   4.2605,   2.4579,  -1.2077,  11.3072,
          8.5219], grad_fn=<AddBackward0>)

In [20]:
labels = ['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
       'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
       'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
       'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee']
output = labels[torch.argmax(output)]

In [21]:
output

'rice'